In [1]:
'''Markowitz Optimization in python, using pandas, and Yahoo Finance Data. Input a 
list of tickers, and the program will calcuate historical returns, a covariance matrix, and 
finally, the individual stock weigthings that maximize the portfolio sharpe ratio, defined as
sharpe ratio = (expected portfolio return - risk free rate)/portfolio standard deviation'''



#various pandas, numpy
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime
import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats
import matplotlib.mlab as mlab
# plotting

import matplotlib.pyplot as plt

# make plots inline
%matplotlib inline

# formatting options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 10) 
pd.set_option('display.width', 82) 
pd.set_option('precision', 7)

In [19]:
'''Create the function that stores Tickers in a dataframe'''
def create_portfolio(tickers, weights=None):
    if weights is None: 
        shares = np.ones(len(tickers))/len(tickers)
    portfolio = pd.DataFrame({'Tickers': tickers, 
                              'Weights': weights}, 
                             index=tickers)
    return portfolio


In [3]:
def get_historical_closes(ticker, start_date, end_date):
    # get the data for the tickers.  This will be a panel
    p = web.DataReader(ticker, "yahoo", start_date, end_date)    
    # convert the panel to a DataFrame and selection only Adj Close
    # while making all index levels columns
    d = p.to_frame()['Adj Close'].reset_index()
    # rename the columns
    d.rename(columns={'minor': 'Ticker', 
                      'Adj Close': 'Close'}, inplace=True)
    # pivot each ticker to a column
    pivoted = d.pivot(index='Date', columns='Ticker')
    # and drop the one level on the columns
    pivoted.columns = pivoted.columns.droplevel(0)
    return pivoted

In [4]:
closes = get_historical_closes(['MSFT', 'AAPL', 'KO', "JPM", "BRK-B", "T"], '2010-01-01', '2017-01-01')
closes[:5]

Ticker           AAPL      BRK-B        JPM         KO       MSFT          T
Date                                                                        
2010-01-04  27.727039  66.220001  36.148636  23.123276  25.555485  19.299578
2010-01-05  27.774976  66.540001  36.848832  22.843558  25.563741  19.205039
2010-01-06  27.333178  66.199997  37.051297  22.835451  25.406859  18.924023
2010-01-07  27.282650  66.459999  37.785239  22.778696  25.142634  18.711547
2010-01-08  27.464034  66.440002  37.692441  22.357094  25.316031  18.574467

In [5]:
def calc_daily_returns(closes):
    return np.log(closes/closes.shift(1))

In [6]:
daily_returns = calc_daily_returns(closes)
daily_returns[:5]

Ticker           AAPL      BRK-B        JPM         KO       MSFT          T
Date                                                                        
2010-01-04        NaN        NaN        NaN        NaN        NaN        NaN
2010-01-05  0.0017274  0.0048207  0.0191847 -0.0121706  0.0003230 -0.0049105
2010-01-06 -0.0160342 -0.0051229  0.0054794 -0.0003550 -0.0061558 -0.0147405
2010-01-07 -0.0018503  0.0039198  0.0196152 -0.0024885 -0.0104542 -0.0112914
2010-01-08  0.0066263 -0.0003009 -0.0024590 -0.0186820  0.0068729 -0.0073529

In [7]:

# calculate annual returns
def calc_annual_returns(daily_returns):
    grouped = np.exp(daily_returns.groupby(
        lambda date: date.year).sum())-1
    return grouped

In [8]:
annual_returns = calc_annual_returns(daily_returns)
annual_returns

Ticker       AAPL      BRK-B        JPM         KO       MSFT          T
2010    0.5072194  0.2097554 -0.0062920  0.1893662 -0.0794414  0.0945481
2011    0.2555803 -0.0475596 -0.1991802  0.0945864 -0.0451566  0.0902392
2012    0.3256689  0.1756225  0.3618060  0.0652759  0.0579886  0.1748770
2013    0.0806949  0.3217392  0.3673336  0.1723302  0.4429798  0.0972209
2014    0.4062250  0.2664473  0.0988281  0.0526609  0.2756462  0.0065947
2015   -0.0301371 -0.1206127  0.0837256  0.0513985  0.2269185  0.0831979
2016    0.1248042  0.2343230  0.3453639 -0.0035709  0.1507775  0.2987440

In [9]:
def calc_portfolio_var(returns, weights=None):
    if weights is None: 
        weights = np.ones(returns.columns.size) / \
        returns.columns.size
    sigma = np.cov(returns.T,ddof=0)
    var = (weights * sigma * weights.T).sum()
    return var

In [10]:
# calculate our portfolio variance (equal weighted)
calc_portfolio_var(annual_returns)

0.0057301736886791509

In [11]:
def sharpe_ratio(returns, weights = None, risk_free_rate = 0.015):
    n = returns.columns.size
    if weights is None: weights = np.ones(n)/n
    # get the portfolio variance
    var = calc_portfolio_var(returns, weights)
    # and the means of the stocks in the portfolio
    means = returns.mean()
    # and return the sharpe ratio
    return (means.dot(weights) - risk_free_rate)/np.sqrt(var)

In [12]:
# calculate equal weighted sharpe ratio
sharpe_ratio(annual_returns)

1.7703630978790612

In [13]:
# function to minimize
def y_f(x): return 2+x**2

In [14]:
scopt.fmin(y_f,1000)

Optimization terminated successfully.
         Current function value: 2.000000
         Iterations: 27
         Function evaluations: 54


array([ 0.])

In [15]:
def negative_sharpe_ratio_n_minus_1_stock(weights, 
                                          returns, 
                                          risk_free_rate):
    """
    Given n-1 weights, return a negative sharpe ratio
    """
    weights2 = sp.append(weights, 1-np.sum(weights))
    return -sharpe_ratio(returns, weights2, risk_free_rate)

In [16]:
def optimize_portfolio(returns, risk_free_rate):
    """ 
    Performs the optimization
    """
    # start with equal weights
    w0 = np.ones(returns.columns.size-1, 
                 dtype=float) * 1.0 / returns.columns.size
    # minimize the negative sharpe value
    w1 = scopt.fmin(negative_sharpe_ratio_n_minus_1_stock, 
                    w0, args=(returns, risk_free_rate))
    # build final set of weights
    final_w = sp.append(w1, 1 - np.sum(w1))
    # and calculate the final, optimized, sharpe ratio
    final_sharpe = sharpe_ratio(returns, final_w, risk_free_rate)
    return (final_w, final_sharpe)

In [17]:
# optimize our portfolio
optimize_portfolio(annual_returns, 0.015)

Optimization terminated successfully.
         Current function value: -2.682138
         Iterations: 169
         Function evaluations: 276


(array([ 0.33181466,  0.03873414,  0.03506712,  0.31419176,  0.06624746,
         0.21394486]), 2.6821379382805546)